In [97]:
from google import genai
from google.genai import types

client = genai.Client()
messages = []

In [98]:
get_weather_function = {
        "name": "get_weather",
        "description": "해당 도시의 날씨를 조회하는 함수",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 얻으려는 도시 이름"
                }
            },
            "required": ["city"]
        }
    }

# Configure the client and tools
tools = types.Tool(function_declarations=[get_weather_function])
config = types.GenerateContentConfig(tools=[tools])

In [ ]:
from google.genai import types

def get_weather(city):
#     """해당 도시의 날씨를 조회하는 함수"""
#     # 실제 구현은 여기에 추가 (예: API 호출)
    return f"{city}로 get_weather함수 호출했습니다!!"

# FunctionMap: 함수 이름과 실제 함수 객체를 매핑
FUNCTION_MAP = {
    "get_weather": get_weather,
    # 새로운 함수를 추가할 때마다 여기에 등록
}

def call_ai():
    # Gemini API는 OpenAI 형식의 messages를 받지 않음
    # 대화 히스토리를 하나의 문자열로 합쳐서 전달
    conversation = ""
    # 시스템 프롬프트 추가: 일반적인 질문에도 답변하도록 명확히 지시
    conversation += "당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 답변해주세요.\n"
    conversation += "- 일반적인 지식 질문에는 자신의 지식을 바탕으로 답변해주세요.\n"
    conversation += "- 날씨 정보가 필요한 경우에만 get_weather 함수를 사용하세요.\n"
    conversation += "- 함수를 사용할 필요가 없는 일반적인 질문에는 직접 답변해주세요.\n\n"
    
    for msg in messages:
        role = msg.get("role", "user")
        content = msg.get("content", "")
        function_call = msg.get("function_call")
        
        if role == "user":
            conversation += f"사용자: {content}\n\n"
        elif role == "assistant":
            if function_call:
                # 함수 호출 요청이 있는 경우
                func_name = function_call.get("name", "")
                func_args = function_call.get("arguments", {})
                if content:
                    conversation += f"AI: {content}\n\n"
                conversation += f"AI가 {func_name} 함수를 호출하려고 합니다. 인자: {func_args}\n\n"
            else:
                conversation += f"AI: {content}\n\n"
        elif role == "function":
            # 함수 실행 결과도 대화에 포함
            function_name = msg.get("name", "")
            conversation += f"함수 {function_name} 실행 결과: {content}\n\n"
    
    # 마지막 메시지가 function role이면 tools 없이 호출 (함수 결과를 받은 후 텍스트 응답)
    # 그렇지 않으면 tools 포함 (새로운 함수 호출 가능)
    last_msg_role = messages[-1].get("role") if messages else None
    use_tools = (last_msg_role != "function")
    
    # 함수 결과를 받은 후에는 명확한 지시 추가
    if not use_tools:
        conversation += "위 함수 실행 결과를 바탕으로 사용자에게 자연스러운 텍스트로 응답해주세요. 더 이상 함수를 호출하지 마세요.\n\n"
    
    # 마지막 사용자 메시지만 전달 (대화 히스토리 포함)
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=conversation,
            config=config if use_tools else None,  # 함수 결과를 받은 후에는 tools 없이 호출
        )
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return

    # response 형태 확인
    # print()   
    # print(response)
    # print()

    # 응답을 messages에 추가
    # messages.append({"role": "assistant", "content": response.text})
    # print(f"AI: {response.text}")
    
    # 텍스트 부분만 추출 (경고 방지)
    parts = response.candidates[0].content.parts
    text_parts = []
    function_call = None
    
    for part in parts:
        if hasattr(part, 'text') and part.text:
            text_parts.append(part.text)
        if hasattr(part, 'function_call') and part.function_call:
            function_call = part.function_call
    
    # Check for a function call
    if function_call:
        function_name = function_call.name
        function_args = function_call.args
        
        print(f"Function to call: {function_name}")
        print(f"Arguments: {function_args}")
        
        # assistant의 함수 호출 요청을 messages에 저장 (중요!)
        # 텍스트 부분이 있으면 함께 저장
        assistant_content = {
            "role": "assistant",
            "content": None,
            "function_call": {
                "name": function_name,
                "arguments": function_args
            }
        }
        if text_parts:
            assistant_content["content"] = "".join(text_parts)
        messages.append(assistant_content)
        
        # FunctionMap에서 함수 찾기 및 실행
        if function_name in FUNCTION_MAP:
            try:
                # 함수 실행
                function = FUNCTION_MAP[function_name]
                result = function(**function_args)
                
                print(f"Function result: {result}")
                
                # 함수 실행 결과를 messages에 추가
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": str(result)
                })
                
                # 함수 결과를 받아서 다시 모델에 전달 (재귀 호출)
                call_ai()
                return
            except Exception as e:
                error_msg = f"함수 실행 중 오류 발생: {str(e)}"
                print(f"Error: {error_msg}")
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": error_msg
                })
                call_ai()
                return
        else:
            error_msg = f"알 수 없는 함수: {function_name}"
            print(f"Error: {error_msg}")
            messages.append({
                "role": "function",
                "name": function_name,
                "content": error_msg
            })
            call_ai()
            return
    else:
        # 텍스트 부분만 합쳐서 사용 (일반 대화)
        response_text = "".join(text_parts)
        if response_text:  # 응답이 있는 경우에만 추가
            messages.append({"role": "assistant", "content": response_text})
            print(f"AI: {response_text}")
        else:
            # 응답이 비어있는 경우 (함수 결과를 받은 후 재귀 호출에서 발생할 수 있음)
            # 마지막 함수 결과를 직접 사용자에게 전달
            if last_msg_role == "function" and messages:
                last_function_result = messages[-1].get("content", "")
                if last_function_result:
                    response_text = f"함수 실행 결과: {last_function_result}"
                    messages.append({"role": "assistant", "content": response_text})
                    print(f"AI: {response_text}")
                else:
                    print("AI: (응답 없음)")
            else:
                print("AI: (응답 없음)")

In [100]:
while True:
    ask = input("send a msg to the LLM: ")
    if ask == 'q' or ask == 'ㅂ':
        break   
    else:
        messages.append({"role": "user", "content": ask})
        print(f"User: {ask}")
        call_ai()
        print()  # 빈 줄 추가


User: 마지막으로 테스트 한다.
AI: 네, 말씀해주세요! 어떤 것을 도와드릴까요?

User: 내이름은 길재용
AI: 반갑습니다, 길재용님! 무엇을 도와드릴까요?


User: 내 이름 뭐라고?
AI: 길재용님이십니다!

User: LG CNS 는 뭐하는 회사여?
AI: LG CNS는 대한민국의 IT 서비스 기업입니다. 주로 시스템 통합(SI), IT 컨설팅, 아웃소싱 등 다양한 IT 서비스를 제공하며, 기업들의 디지털 전환을 돕는 역할을 합니다.

User: 서울 날씨는?
Function to call: get_weather
Arguments: {'city': '서울'}
Function result: 서울로 get_weather함수 호출했습니다!!
AI: 네, 서울 날씨 정보를 확인하고 있습니다!

User: 대한 민국 위치는?
AI: 대한민국은 동아시아에 위치한 나라로, 한반도의 남쪽에 있습니다. 서쪽으로는 황해를 사이에 두고 중국과 마주하고, 동쪽으로는 동해를 사이에 두고 일본과 마주하고 있습니다. 북쪽으로는 비무장지대(DMZ)를 경계로 북한과 접하고 있습니다.

User: 내 이름 다시 한번 뭐라고?
AI: 길재용님이십니다!

User: 굳
AI: 네, 감사합니다! 더 궁금한 점이나 필요한 것이 있으시면 언제든지 말씀해주세요.

